# Madripiso accounts

In this Jupyter notebook, you will process an Excel file with all the items which have been bought in the Madripiso, to know how many money everybody owes to everybody.

## Step 1: Update the Madripiso accounts' Excel file

Update and download the Excel file from Google Drive' **Madripiso** folder. If you do not live in Madripiso, you will find a sample Excel file in my GitHub repository called Madripiso (**sample.xlsx**).

This Excel file contains seven columns:

- **Qué**: this column contains the name of each item (string).
- **Quién**: this column contains the initial letter of the name of the person who has bought the **Qué** item (one-character string).
- **Para quién**: this column shows the initial letter of the names of people who will use (eat, drink, or whatever) the **Qué** item (string).
- **Cuánto**: this column indicates the amount of money that **Quién** spent on **Qué** item (two-decimals float).
- **Cuándo**: this column contains the date in which the item was bought; format: yyyy-mm-dd (date).
- **Dónde**: this column shows the place where the item was bought (string).
- **Incidencias**: this column indicates any unforeseen circumstance that has happened when annotating the item (string).

Initially, to process all accounts, only **Quién**, **Para quién** and **Cuánto** have been used.

## Step 2: Write the correct path to the Madripiso accounts' Excel file

Now, between both quotation marks, write the path to the Excel file in your computer:

In [107]:
import pandas as pd
path = "C:/Users/edunu/Downloads/gastos-del-piso-hasta-octubre.xlsx"

## Step 3: Load the Excel file into Python

In the previous step, we imported *pandas* package. This is a very useful package to deal with dataframes, which is the aim of this notebook.

We load the Excel file into Python, using the path written in the previous step. The object with Excel data is called **gastos**.

In [108]:
gastos = pd.read_excel(path)
gastos

,Qué,Quién,ParaQuién,Cuánto,Cuándo,Dónde,Unnamed: 6
0,Yogur fresa 0%,E,EJ,1.12,2020-08-28,Mercadona,NaN
1,Pan integral,E,EJ,0.95,2020-08-28,Mercadona,NaN
2,Leche semi,E,EJ,3.48,2020-08-28,Mercadona,NaN
3,Tomillo,E,EJ,0.69,2020-08-28,Mercadona,NaN
4,Filete de pollo,E,EJX,5.25,2020-08-28,Mercadona,NaN
...,...,...,...,...,...,...,...
192,Muesli Choco,J,JL,1.75,2020-09-28,Mercadona,NaN
193,Queso lonchas Edam,J,AJLNX,1.70,2020-09-28,Mercadona,NaN
194,Torti pata cebolla,J,AJLNX,1.85,2020-09-28,Mercadona,NaN
195,Leche Semi sin lactosa,J,A,0.75,2020-09-28,Mercadona,NaN


## Step 4: Create a database with all accounts in Madripiso

The objective of this block is to create a dataframe with the same number of rows and columns (squared). They will contain the money that everybody owes to a given person (who gives name to the row), and the money that a given person owes to everybody (who gives name to the column). For example:

|       | **C** | **E** | **R** |
|-------|-------|-------|-------|
| **C** | 0     | 5     | 0     |
| **E** | 0     | 0     | 2     |
| **R** | 0     | 0     | 0     | 

This dataframe, similar to the dataframe we are generating, shows that Eduardo (it's me, hi everybody!) owes 5 dollars to Charlie (C) and Rose (R) owes 2 dollars to Eduardo.

First, we are creating a dictionary which links each initial to its corresponding name. If you live in Madripiso, execute the following code cell:

In [109]:
nombres = {"A": "A", "E": "Eduardo", "J": "J", "L": "L",
           "N": "N", "X": "Foreigner"}


In order to protect my colleagues' identities, I'm not including their names in this public Jupyter notebook but I'm sure they will know which name corresponds to each initial letter.

On the other hand, if you are not from Madripiso and you are using **sample.xlsx**, execute this other code cell:

In [110]:
# nombres = {"C": "Charles", "E": "Eduardo", "R": "Rose"}

The presence of **Foreigner** in the dataframe will help in adjusting accounts when a person who does not live in Madripiso uses an item which somebody in the Madripiso bought. In example: if **Charles** buys an Spanish omelette for dinner and this Spanish omelette is eaten by **Rose** and three **Foreigners** during dinner, it is not fair to say that **Rose** owes to **Charles** the price of the whole Spanish omelette, but only the proportional part (in this case, 25% of the price).

To generate the final dataframe, I am creating a big dictionary (**tabla**) comprised of small dictionaries (**deuda**). Each **deuda** dictionary contains the sum of the money that every person (value) owes to a given person (key). This is reached by appending to a list (**deudita**) all owings from a given person, computing the sum of all this values and linking this sum to a letter. This is the code: 

In [111]:
tabla = dict()
for name in list(nombres.keys())[:-1]:
    deuda = dict()
    for nombre in list(nombres.keys())[:-1]:
        deudita = list()
        for _ in range(len(gastos)):
            condition1 = name != nombre 
            condition2 = name in gastos["ParaQuién"][_]
            condition3 = nombre in gastos["Quién"][_]
            if condition1 and condition2 and condition3:
                for i in range(gastos["ParaQuién"][_].count(name)):
                    deudita.append(round(gastos["Cuánto"][_]/len(gastos["ParaQuién"][_]), 2))
        deuda[nombre] = sum(deudita)
    tabla[name] = deuda

del deudita, deuda, gastos, name, nombre, i
tabla = pd.DataFrame.from_dict(tabla, orient = "index").transpose()

tabla

,A,E,J,L,N
A,0.00,0.50,3.24,9.11,5.45
E,3.29,0.00,64.60,3.29,3.29
J,10.29,64.40,0.00,10.63,6.27
L,10.69,0.00,10.81,0.00,0.00
N,6.90,1.41,6.90,7.07,0.00


## Step 5: Print outputs

The final step is to print some outputs!! All outputs are written in Spanish, because we prefer it.

### First: how many money owes each person to any other person?

Let's see!

In [112]:
print("#######################################\n" +
      "# ESTAS SON LAS CUENTAS DEL MADRIPISO #\n" +
      "#######################################\n")

print("¿Cuánto debe cada uno a cada uno?\n")
for name in list(nombres.keys())[:-1]:
    for nombre in list(nombres.keys())[:-1]:
        if name != nombre and tabla[nombre][name] != 0:
            print("%s debe %.2f euros a %s." % (nombres[nombre], 
                                          tabla[nombre][name],
                                          nombres[name]))

#######################################
# ESTAS SON LAS CUENTAS DEL MADRIPISO #
#######################################

¿Cuánto debe cada uno a cada uno?

Eduardo debe 0.50 euros a A.
J debe 3.24 euros a A.
L debe 9.11 euros a A.
N debe 5.45 euros a A.
A debe 3.29 euros a Eduardo.
J debe 64.60 euros a Eduardo.
L debe 3.29 euros a Eduardo.
N debe 3.29 euros a Eduardo.
A debe 10.29 euros a J.
Eduardo debe 64.40 euros a J.
L debe 10.63 euros a J.
N debe 6.27 euros a J.
A debe 10.69 euros a L.
J debe 10.81 euros a L.
A debe 6.90 euros a N.
Eduardo debe 1.41 euros a N.
J debe 6.90 euros a N.
L debe 7.07 euros a N.


### Second: how many money owes each person to any other person, in a simplified way?

Here, we are calculating the money that person A owes to person B minus the money that person B owes to person A. Also, all foreigners accounts are excluded.

In [113]:
print("\n\n\n" +
      "#################\n" +
      "# SIMPLIFICANDO #\n" +
      "#################\n")

print("Pero, haciendo cuentas, ¿cuánto debe realmente\n" +
      "cada uno a cada uno?\n")

k = len(list(nombres.keys())[:-1])
i = 0 # Esto es un contador para evitar repeticiones 
for name in list(nombres.keys()):

    for nombre in list(nombres.keys())[i:k]:
        if tabla[nombre][name] != 0 or tabla[name][nombre] != 0:
            if nombre != "X" and name != "X":
                if tabla[nombre][name] > tabla[name][nombre]:
                    print("%s debe %.2f euros a %s." % (nombres[nombre], 
                                              tabla[nombre][name] - tabla[name][nombre],
                                              nombres[name]))
                elif tabla[name][nombre] > tabla[nombre][name]:
                    print("%s debe %.2f euros a %s." % (nombres[name], 
                                              tabla[name][nombre] - tabla[nombre][name],
                                              nombres[nombre]))
    i +=1




#################
# SIMPLIFICANDO #
#################

Pero, haciendo cuentas, ¿cuánto debe realmente
cada uno a cada uno?

A debe 2.79 euros a Eduardo.
A debe 7.05 euros a J.
A debe 1.58 euros a L.
A debe 1.45 euros a N.
J debe 0.20 euros a Eduardo.
L debe 3.29 euros a Eduardo.
N debe 1.88 euros a Eduardo.
J debe 0.18 euros a L.
J debe 0.63 euros a N.
L debe 7.07 euros a N.


### Third: this is for the lazy ones

In this piece of code, Python only shows you the money that a given person owes to anybody, and the money that anybody owes to a given person. You have to enter the initial letter of the name of this given person (capital or small letter, it works the same way).

In [114]:
print("\n\n\n" +
      "######################\n" +
      "# PARA LOS PEREZOSOS #\n" +
      "######################\n")

print("¿Te da pereza buscar tu nombre entre tanta línea?")

inicial = input("Dime cuál es tu inicial y te lo pondré más fácil: ")
print("")
if inicial in "aejlnAEJLN":
    inicial = inicial.upper()
    k = len(list(nombres.keys())[:-1])
    i = 0 # Esto es un contador para evitar repeticiones como "Avelina debe 0 a Avelina"
    for name in list(nombres.keys())[:-1]:
        i +=1
        for nombre in list(nombres.keys())[i:k]:
            if tabla[nombre][name] != 0 or tabla[name][nombre] != 0:
                    if nombre == inicial or name == inicial:
                        if tabla[nombre][name] > tabla[name][nombre]:
                            print("%s debe %.2f euros a %s." % (nombres[nombre], 
                                                      tabla[nombre][name] - tabla[name][nombre],
                                                      nombres[name]))
                        elif tabla[name][nombre] > tabla[nombre][name]:
                            print("%s debe %.2f euros a %s." % (nombres[name], 
                                                      tabla[name][nombre] - tabla[nombre][name],
                                                      nombres[nombre]))    
del name  , inicial, i, k




######################
# PARA LOS PEREZOSOS #
######################

¿Te da pereza buscar tu nombre entre tanta línea?
Dime cuál es tu inicial y te lo pondré más fácil: e

A debe 2.79 euros a Eduardo.
J debe 0.20 euros a Eduardo.
L debe 3.29 euros a Eduardo.
N debe 1.88 euros a Eduardo.


In [115]:
tabla

,A,E,J,L,N
A,0.00,0.50,3.24,9.11,5.45
E,3.29,0.00,64.60,3.29,3.29
J,10.29,64.40,0.00,10.63,6.27
L,10.69,0.00,10.81,0.00,0.00
N,6.90,1.41,6.90,7.07,0.00


### Last one: what is the total money that each person is owed, or the total money that each person owes?

In [116]:
print("\n\n\n###########\n" +
      "# TOTALES #\n" +
      "###########\n")            
print("En total, ¿cuánto debe cada uno, o cuánto\n" +
      "se le debe a cada uno?\n")
tabla = tabla[:5]
for nombre in list(nombres.keys())[:-1]:
    debe = pd.DataFrame.sum(tabla, axis = 0)[nombre]
    se_le_debe = pd.DataFrame.sum(tabla, axis = 1)[nombre]
    total = debe - se_le_debe
    if nombre != "X":
        if total > 0:
            print("%s: debe %.2f euros." % (nombres[nombre], 
                                      total))
        elif total < 0:
            print("%s: se le deben %.2f euros." % (nombres[nombre], 
                                      -1 * total))  
del nombre, debe, se_le_debe, total, nombres




###########
# TOTALES #
###########

En total, ¿cuánto debe cada uno, o cuánto
se le debe a cada uno?

A: debe 12.87 euros.
Eduardo: se le deben 8.16 euros.
J: se le deben 6.04 euros.
L: debe 8.60 euros.
N: se le deben 7.27 euros.
